# ETL Pipeline Preparation
Follow the instructions below to help you create your ETL pipeline.
### 1. Import libraries and load datasets.
- Import Python libraries
- Load `messages.csv` into a dataframe and inspect the first few lines.
- Load `categories.csv` into a dataframe and inspect the first few lines.

In [1]:
# import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

In [2]:
# load messages dataset
messages = pd.read_csv('./messages.csv')
messages.head()

,id,message,original,genre
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct


In [3]:
# load categories dataset
categories = pd.read_csv('./categories.csv')
categories.head()

,id,categories
0,2,related-1;request-0;offer-0;aid_related-0;medi...
1,7,related-1;request-0;offer-0;aid_related-1;medi...
2,8,related-1;request-0;offer-0;aid_related-0;medi...
3,9,related-1;request-1;offer-0;aid_related-1;medi...
4,12,related-1;request-0;offer-0;aid_related-0;medi...


### 2. Merge datasets.
- Merge the messages and categories datasets using the common id
- Assign this combined dataset to `df`, which will be cleaned in the following steps

In [4]:
# merge datasets
df = messages.join(categories.set_index('id'), on='id')
df.head()

,id,message,original,genre,categories
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,related-1;request-0;offer-0;aid_related-0;medi...
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,related-1;request-0;offer-0;aid_related-1;medi...
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,related-1;request-0;offer-0;aid_related-0;medi...
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,related-1;request-1;offer-0;aid_related-1;medi...
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,related-1;request-0;offer-0;aid_related-0;medi...


### 3. Split `categories` into separate category columns.
- Split the values in the `categories` column on the `;` character so that each value becomes a separate column. You'll find [this method](https://pandas.pydata.org/pandas-docs/version/0.23/generated/pandas.Series.str.split.html) very helpful! Make sure to set `expand=True`.
- Use the first row of categories dataframe to create column names for the categories data.
- Rename columns of `categories` with new column names.

In [5]:
# make dataframe with only values from the categories column
# convert to int
df_wip = df.categories.str.split(';', expand = True)
for col in df_wip:
    df_wip[col] = df_wip[col].str[-1].astype('int')

In [6]:
# get column names from category column
cols = [i.split('-')[0] for i in categories['categories'][0].split(';')]

In [7]:
# assign new column names
df_wip.columns = cols

### 5. Replace `categories` column in `df` with new category columns.
- Drop the categories column from the df dataframe since it is no longer needed.
- Concatenate df and categories data frames.

In [8]:
# append work in progress dataframe and the df
df = pd.concat([df, df_wip], axis = 1)

In [9]:
# drop categories column
df.drop(columns = ['categories'], inplace = True)

### 6a. Remove duplicates.
- Check how many duplicates are in this dataset.
- Drop the duplicates.
- Confirm duplicates were removed.

In [10]:
# how many duplicated rows
print(str(df.duplicated().sum()) + ' dupliacted rows')

#drop dupliacted rows
df.drop_duplicates(inplace = True)

# check again to verify duplicates are dropped
print(str(df.duplicated().sum()) + ' dupliacted rows')

170 dupliacted rows
0 dupliacted rows


In [11]:
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 6b. Check Dummy Variables

All columns should have 0 and 1 for the newly created columns with dummy variables. Check all newly created columns for this logic. Check output columns for errors in dummy variables

In [12]:
test = list(df.columns)
test[4:]

#df['related']


for i in test[4:]:
    if len(df[i].unique()) != 2:
        print(i + ': ' + str(df[i].unique()))

related: [1 0 2]
child_alone: [0]


#### 'related' column

- Expected 0's and 1's, so explore column to see if we can see if 2's represent anything

In [13]:
df.iloc[:,4].value_counts()

# get counts for each category in the 'related column'
df.iloc[:,4].value_counts()

1    19906
0     6122
2      188
Name: related, dtype: int64

In [14]:
# explore all data that had 2 in the 'related' column
for i in df.iloc[:,4:].columns:
    print(df[df['related'] == 2][i].value_counts())
# result: have values of 0 for the rest of the columns

2    188
Name: related, dtype: int64
0    188
Name: request, dtype: int64
0    188
Name: offer, dtype: int64
0    188
Name: aid_related, dtype: int64
0    188
Name: medical_help, dtype: int64
0    188
Name: medical_products, dtype: int64
0    188
Name: search_and_rescue, dtype: int64
0    188
Name: security, dtype: int64
0    188
Name: military, dtype: int64
0    188
Name: child_alone, dtype: int64
0    188
Name: water, dtype: int64
0    188
Name: food, dtype: int64
0    188
Name: shelter, dtype: int64
0    188
Name: clothing, dtype: int64
0    188
Name: money, dtype: int64
0    188
Name: missing_people, dtype: int64
0    188
Name: refugees, dtype: int64
0    188
Name: death, dtype: int64
0    188
Name: other_aid, dtype: int64
0    188
Name: infrastructure_related, dtype: int64
0    188
Name: transport, dtype: int64
0    188
Name: buildings, dtype: int64
0    188
Name: electricity, dtype: int64
0    188
Name: tools, dtype: int64
0    188
Name: hospitals, dtype: int64
0    188
Name: sho

In [15]:
# what do rows with value of 0 in the related column
# have in the other columns?
for i in df.iloc[:,4:].columns:
    print(df[df['related'] == 0][i].value_counts())
#result: similar to 2. values of 0 for the rest of the columns

0    6122
Name: related, dtype: int64
0    6122
Name: request, dtype: int64
0    6122
Name: offer, dtype: int64
0    6122
Name: aid_related, dtype: int64
0    6122
Name: medical_help, dtype: int64
0    6122
Name: medical_products, dtype: int64
0    6122
Name: search_and_rescue, dtype: int64
0    6122
Name: security, dtype: int64
0    6122
Name: military, dtype: int64
0    6122
Name: child_alone, dtype: int64
0    6122
Name: water, dtype: int64
0    6122
Name: food, dtype: int64
0    6122
Name: shelter, dtype: int64
0    6122
Name: clothing, dtype: int64
0    6122
Name: money, dtype: int64
0    6122
Name: missing_people, dtype: int64
0    6122
Name: refugees, dtype: int64
0    6122
Name: death, dtype: int64
0    6122
Name: other_aid, dtype: int64
0    6122
Name: infrastructure_related, dtype: int64
0    6122
Name: transport, dtype: int64
0    6122
Name: buildings, dtype: int64
0    6122
Name: electricity, dtype: int64
0    6122
Name: tools, dtype: int64
0    6122
Name: hospitals, dtype:

In [16]:
# what do rows with value of 1 in the related column
# have in the other columns?
for i in df.iloc[:,4:].columns:
    print(df[df['related'] == 1][i].value_counts())
# result: mix of 1's and 0's for rest of columns

1    19906
Name: related, dtype: int64
0    15432
1     4474
Name: request, dtype: int64
0    19788
1      118
Name: offer, dtype: int64
1    10860
0     9046
Name: aid_related, dtype: int64
0    17822
1     2084
Name: medical_help, dtype: int64
0    18593
1     1313
Name: medical_products, dtype: int64
0    19182
1      724
Name: search_and_rescue, dtype: int64
0    19435
1      471
Name: security, dtype: int64
0    19046
1      860
Name: military, dtype: int64
0    19906
Name: child_alone, dtype: int64
0    18234
1     1672
Name: water, dtype: int64
0    16983
1     2923
Name: food, dtype: int64
0    17592
1     2314
Name: shelter, dtype: int64
0    19501
1      405
Name: clothing, dtype: int64
0    19302
1      604
Name: money, dtype: int64
0    19608
1      298
Name: missing_people, dtype: int64
0    19031
1      875
Name: refugees, dtype: int64
0    18712
1     1194
Name: death, dtype: int64
0    16460
1     3446
Name: other_aid, dtype: int64
0    18201
1     1705
Name: infrastruc

Columns with values of 2 in the 'related' column have 0's in the remaining columns. This behaves similarly with rows that had 0 in the 'related' column

Therefore, will reclassify values of 2 as 0 since we are performing binary categorization and these two categories behave similarly.

For future reference, look into what a 2 means, why it was labeled with 2, and why it happened.

In [17]:
# following is a test to see if logic works for replacing 2's with 0's.
# created a dummy data frame to test, made new column, and
# compared original values with replaced values.
# rows with 2 in the original 'related' column should 
# should have all 0's in the value_counts() output
# uncomment below if want to see test

#df2 = df
#df2['related_V2'] = df['related']
#df2['related_V2'].replace({2: 0}, inplace = True)
#df2[df2['related'] == 2]['related_V2'].value_counts()

In [18]:
# replace 2's with 0's in dataframe
df['related'].replace({2: 0}, inplace = True)

# test to see if all 0's and 1's
df['related'].value_counts()

1    19906
0     6310
Name: related, dtype: int64

#### 'child_alone' column

- This column has all 0's for values, which seems odd since should be 0's and ones
- However, will leave this alone since there are some algorithms that can handle a single label. Removing this column seems inappropriate at the moment since this can still tell us something about the data

### 7. Save the clean dataset into an sqlite database.
You can do this with pandas [`to_sql` method](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.to_sql.html) combined with the SQLAlchemy library. Remember to import SQLAlchemy's `create_engine` in the first cell of this notebook to use it below.

In [27]:
engine = create_engine('sqlite:///DisasterResponse.db')
df.to_sql('disaster_resp', engine, if_exists = 'replace', index=False)

### 8. Use this notebook to complete `etl_pipeline.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database based on new datasets specified by the user. Alternatively, you can complete `etl_pipeline.py` in the classroom on the `Project Workspace IDE` coming later.